# Autoencoders

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import neuralnetworksA3 as nn

## Climate Change

Patterns in [global temperature data](https://www.cs.colostate.edu/~anderson/cs545/notebooks/global_temps.npz) provide a way to investigate climate change.  This data is from the [Coupled Model Intercomparison Project Phase 5](https://pcmdi.llnl.gov/mips/cmip5/)

In [ ]:
temps = np.load('global_temps.npz')
temps.files

In [ ]:
data = temps['data']
lats = temps['lats']
lons = temps['lons']

data.shape, lats.shape, lons.shape

The data was produced by 29 different simulations of the dynamics of the earth's atmosphere.  The simulations produced global temperatures for 180 years, from 1920 through 2100.

We are using the `cartopy` package.  You should be able to install it in your anaconda distribution using

      conda install -c anaconda cartopy

In [ ]:
import cartopy.util as cutil
import cartopy.crs as ccrs
import cartopy.feature as cpf
from mpl_toolkits.axes_grid1 import make_axes_locatable  # for colorbar

def draw_on_globe(data, lats, lons, cmap='coolwarm', vmin=None, vmax=None,
                fig=None, axes=None):
    if fig is None:
        fig = plt.figure()
        
    if axes is None:
        axes = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

    data_cyclic, lons_cyclic = cutil.add_cyclic_point(data, coord=lons)

    image = axes.pcolormesh(lons_cyclic, lats, data_cyclic,
                          cmap=cmap, vmin=vmin, vmax=vmax,
                          transform=ccrs.PlateCarree() )

    axes.coastlines();
    
    divider = make_axes_locatable(axes)
    ax_cb = divider.new_horizontal(size="5%", pad=0.1, axes_class=plt.Axes)

    plt.gcf().add_axes(ax_cb)
    plt.colorbar(image, cax=ax_cb)

    plt.sca(axes)  # in case other calls, like plt.title(...), will be made
    # return image


In [ ]:
data.shape

In [ ]:
data = data - data.mean((2, 3))[:, :, np.newaxis, np.newaxis]

In [ ]:
draw_on_globe(data[0, 0, :, :], lats, lons)
plt.title('First Simulation, Year 1920');

In [ ]:
draw_on_globe(data[0, 179, :, :], lats, lons)
plt.title('First Simulation, Year 2100');

In [ ]:
temperature_differences = data[0, 179, :, :] - data[0, 0, :, :]
draw_on_globe(temperature_differences, lats, lons)
plt.title('First Simulation, Year 2100 - 1920');

Let's swap the first two axes, so the first dimension is the years.  Then we will use the different years as different samples and try to project data into two dimensions and see how the years fall in the two-dimensional plane.

In [ ]:
np.moveaxis?

In [ ]:
data.shape

In [ ]:
data_shifted = np.moveaxis(data, 0, 1)
data.shape, data_shifted.shape

To save time, just use data from the first two (of 29) models.

In [ ]:
X = data_shifted[:, 0:2, :, :].reshape(180, -1)
X.shape

In [ ]:
nnet = nn.NeuralNetwork(X.shape[1], [50, 20, 20, 2, 20, 20, 50], X.shape[1])

nnet.train(X, X, 100, 'scg')  # notice the target !
plt.plot(nnet.get_performance_trace());

In [ ]:
nnet.train(X, X, 100, 'scg')  # notice the target !
plt.plot(nnet.get_performance_trace());

In [ ]:
len(nnet.Ys)

In [ ]:
bottle = nnet.Ys[4]
bottle.shape

In [ ]:
plt.plot(bottle[:, 0], bottle[:, 1], '-o')
r = 0.02
for year, x, y in zip(range(1920, 2100), bottle[:, 0], bottle[:, 1]):
    plt.annotate(str(year), xy=(x+np.random.uniform(-r, r),
                                y+np.random.uniform(-r, r)),
                 size=10);

In [ ]:
bottle.shape

In [ ]:
plt.scatter(bottle[:, 0], bottle[:, 1], s=np.arange(180)*0.2, alpha=0.5)
plt.plot(bottle[:, 0], bottle[:, 1], '-', alpha=0.2);

## Robot Kinematics

In [ ]:
from math import pi
import time
import IPython.display as ipd  # for display and clear_output


class Robot():

    def __init__(self, link_lengths):
        self.n_links = len(link_lengths)
        self.link_lengths = np.array(link_lengths)
        self.joint_angles = np.zeros(self.n_links)
        self.points = [[10, 10] for _ in range(self.n_links + 1)]
        self.lim = sum(link_lengths)
        self.update_points()

    def update_joints(self, joint_angles):
        self.joint_angles = joint_angles
        self.update_points()

    def add_to_joints(self, joint_angle_deltas):
        self.joint_angles += joint_angle_deltas
        too_high = self.joint_angles > 2 * pi
        self.joint_angles[too_high] = self.joint_angles[too_high] - 2 * pi
        too_low = self.joint_angles < 0
        self.joint_angles[too_low] = self.joint_angles[too_low] + 2 * pi
        self.update_points()

    def update_points(self):
        for i in range(1, self.n_links + 1):
            self.points[i][0] = self.points[i - 1][0] + \
                self.link_lengths[i - 1] * \
                np.cos(np.sum(self.joint_angles[:i]))
            self.points[i][1] = self.points[i - 1][1] + \
                self.link_lengths[i - 1] * \
                np.sin(np.sum(self.joint_angles[:i]))

        self.end_effector = np.array(self.points[self.n_links]).T

    def get_state(self):
        return np.hstack((np.sin(self.joint_angles),
                          np.cos(self.joint_angles)))

    def plot(self, obstacles=[]):
        for i in range(self.n_links + 1):
            if i is not self.n_links:
                plt.plot([self.points[i][0], self.points[i + 1][0]],
                         [self.points[i][1], self.points[i + 1][1]], 'r-')
            plt.plot(self.points[i][0], self.points[i][1], 'k.')
       
        plt.axis('square')
        plt.xlim([-1, 21])
        plt.ylim([-1, 21])
        # plt.pause(1e-2)

    def illustrate(self):
        for i in range(10):
            action = np.random.uniform(0.1, 0.2, size=self.n_links)
            # action = np.random.choice([0, 0.2, 0, 0.2], size=self.n_links)
            # action = [0, 0, 0, 0.1]
            self.add_to_joints(action)
            self.plot()

In [ ]:
arm = Robot([4., 5., 4., 6.])
arm.illustrate()

In [ ]:
arm = Robot([4., 5., 4., 6.])
graphics = False
points = []
fig = plt.figure()
for i in range(1000):
    action = np.random.uniform(0.1, 0.2, size=arm.n_links)
    arm.add_to_joints(action)
    if graphics:
        plt.clf()
        arm.plot()
        ipd.clear_output(wait=True)
        ipd.display(fig)
        time.sleep(0.2)  # 0.2 seconds
    ipd.clear_output(wait=True)
 
    points.append(arm.points[1] + arm.points[2])

In [ ]:
points = np.array(points)
points.shape

In [ ]:
for p in points[:10]:
    arm.update_joints(p)
    arm.plot()

In [ ]:
X = np.array(points)

nnet = nn.NeuralNetwork(X.shape[1], [100, 10, 2, 50], X.shape[1])
nnet.train(X, X, 5000, 'scg')

plt.figure()
plt.plot(nnet.get_performance_trace())

Y = nnet.use(X)

plt.figure(figsize=(12, 12))
plt.plot(X, '--')
plt.plot(Y)

plt.figure(figsize=(12, 12))
bottle = nnet.Ys[3]
plt.plot(bottle[:, 0], bottle[:, 1], '-o');

In [ ]:
nnet = nn.NeuralNetwork(X.shape[1], [20, 10, 3, 20], X.shape[1])
# nnet = nn.NeuralNetwork(X.shape[1], [100, 100, 50, 3, 50, 100], X.shape[1])
nnet.train(X, X, 4000, 'scg')

plt.figure()
plt.plot(nnet.get_performance_trace())

Y = nnet.use(X)

plt.figure(figsize=(12, 12))
plt.plot(X, '--')
plt.plot(Y)

from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12, 12))
ax = plt.subplot(projection='3d')
bottle = nnet.Ys[3]
plt.plot(bottle[:, 0], bottle[:, 1], bottle[:, 2], '-o', alpha=0.5);

## Rates of Neurons Firing During Movements

From [Neural Latents Benchmark](https://neurallatents.github.io/datasets) site. Download [this data file](https://www.cs.colostate.edu/~anderson/cs545/notebooks/neuron_rates.npz).

In [ ]:
file = np.load('neuron_rates.npz')
rates = file['rates']
colors = file['colors']
rates.shape, colors.shape

In [ ]:
plt.plot(rates[:500, :20]);

In [ ]:
X = rates

In [ ]:
nnet = nn.NeuralNetwork(X.shape[1], [2], X.shape[1])
# nnet = nn.NeuralNetwork(X.shape[1], [100, 100, 50, 3, 50, 100], X.shape[1])
nnet.train(X, X, 4000, 'scg')

In [ ]:
plt.plot(nnet.get_error_trace());

In [ ]:
Y = nnet.use(X)
bottle = nnet.Ys[1]
bottle.shape

In [ ]:
n_trials = 27
bottle = bottle.reshape(n_trials, -1, 2)

In [ ]:
bottle.shape

In [ ]:
plt.figure(figsize=(20,20))
for trial, color in zip(bottle, colors):
    plt.plot(trial[:, 0], trial[:, 1], color=color)
    plt.scatter(trial[0, 0], trial[0, 1], color=color)

Since we are looking for "dynamics", meaning changes in time, let's try predicting the change in firing rates, rather than the firing rates themselves.

In [ ]:
X_trials = X.copy().reshape(n_trials, 100, -1)
X_trials.shape

In [ ]:
X_trials_diff = X_trials[:, 1:, :] - X_trials[:, :-1, :]
X_trials_diff.shape

In [ ]:
X_trials = X_trials[:, :-1, :]
X_trials.shape

In [ ]:
X_trials = X_trials.reshape(-1, 182)
X_trials_diff = X_trials_diff.reshape(-1, 182)
X_trials.shape, X_trials_diff.shape

In [ ]:
nnet = nn.NeuralNetwork(X.shape[1], [2], X.shape[1])
# nnet = nn.NeuralNetwork(X.shape[1], [100, 100, 50, 3, 50, 100], X.shape[1])
nnet.train(X_trials, X_trials_diff, 2000, 'scg')

In [ ]:
plt.plot(nnet.get_performance_trace());

In [ ]:
Y = nnet.use(X)
bottle = nnet.Ys[1]
bottle.shape

In [ ]:
n_trials = 27
bottle = bottle.reshape(n_trials, -1, 2)
bottle.shape

In [ ]:
plt.figure(figsize=(20,20))
for trial, color in zip(bottle, colors):
    plt.plot(trial[:, 0], trial[:, 1], color=color)
    plt.scatter(trial[0, 0], trial[0, 1], color=color)